In [1]:
import urllib.request
import zipfile

url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
output = 'cats_and_dogs_filtered.zip'

# Download
urllib.request.urlretrieve(url, output)

# Unzip
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall()


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [8]:
base_dir = "cats_and_dogs_filtered"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/validation"

IMG_SIZE = (160, 160)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(rescale = 1./255)
val_gen = ImageDataGenerator(rescale = 1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
base_model = MobileNetV2(input_shape = IMG_SIZE + (3,), include_top = False, weights = 'imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:
history = model.fit(
    train_data,
    epochs=5,
    validation_data=val_data,
    verbose = 1
)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 689ms/step - accuracy: 0.8848 - loss: 0.2672

C:\Users\Kaustubh Srivastava\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.8853 - loss: 0.2660 - val_accuracy: 0.9480 - val_loss: 0.1331
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 42s 674ms/step - accuracy: 0.9617 - loss: 0.1227 - val_accuracy: 0.9560 - val_loss: 0.1120
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 43s 679ms/step - accuracy: 0.9739 - loss: 0.0763 - val_accuracy: 0.9570 - val_loss: 0.1050
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 49s 786ms/step - accuracy: 0.9666 - loss: 0.0833 - val_accuracy: 0.9650 - val_loss: 0.0947
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 47s 744ms/step - accuracy: 0.9737 - loss: 0.0668 - val_accuracy: 0.9620 - val_loss: 0.0930


In [13]:
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy after fine-tuning: {acc:.4f}")
model.save("mobilenet_finetuned_model.h5")


32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 503ms/step - accuracy: 0.9639 - loss: 0.0868


Validation Accuracy after fine-tuning: 0.9620
